<a href="https://colab.research.google.com/github/alvarolj23/brujulaDataScientist/blob/main/03_MVP_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# System preparation

## Import libraries

In [ ]:
#pip install scikit-plot

In [1]:
# Graphics
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt
# Data 
import numpy as np
import pandas as pd

#Regular expressions
import re as re

# Utilities
from time import time
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split


# Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Classification
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, f1_score , roc_auc_score


# Classification algorithms
from sklearn.linear_model import LogisticRegression

# Others
import warnings
from functools import reduce
from pathlib import Path
import os



In [2]:
# Configurations.
%matplotlib inline
sns.set_style("darkgrid")
warnings.filterwarnings("ignore")
plt.rc("font", family="serif", size=15)

# MVP ML

### Read data 

In [4]:
compressors_all_data_hourly = pd.read_csv("data/compressors_all_data_hourly.csv", error_bad_lines=False, index_col=0)

In [5]:
compressors_all_data_hourly.head()

,datetime,compressorID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,current,rpm,pressure,vibration,...,error4,error5,comp1_fail,comp2_fail,comp3_fail,comp4_fail,age,model2,model3,model4
0,2014-06-01 06:00:00,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
1,2014-06-01 07:00:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
2,2014-06-01 08:00:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
3,2014-06-01 09:00:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
4,2014-06-01 10:00:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0


### Create the cycle column

We will build a column with the number of the cycle. When building the model we will drop the datetime column and the cycle will help the model understanding the data is following chronological order.

Data is described in the dataset each hour. Hence, it's assumed each cycle is equivalent to 1 hours

In [6]:
# Create a list for the cycles
cycles=[]
len(compressors_all_data_hourly.groupby('compressorID')['datetime'].count())

100

In [7]:
# Loop the whole dataframe by compressorID and fill for each compressorID with the cycle number
for machine in range(1,len(compressors_all_data_hourly.groupby('compressorID')['datetime'].count())+1):
  num_cycle= 0
  for num_cycle in range (1, compressors_all_data_hourly.groupby('compressorID')['datetime'].count()[machine]+1 ):   
    cycles.append(num_cycle)

In [8]:
# Insert column in third position
compressors_all_data_hourly.insert(2, 'cycle', cycles)

In [9]:
compressors_all_data_hourly.head()

,datetime,compressorID,cycle,comp1_maint,comp2_maint,comp3_maint,comp4_maint,current,rpm,pressure,...,error4,error5,comp1_fail,comp2_fail,comp3_fail,comp4_fail,age,model2,model3,model4
0,2014-06-01 06:00:00,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
1,2014-06-01 07:00:00,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
2,2014-06-01 08:00:00,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
3,2014-06-01 09:00:00,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0
4,2014-06-01 10:00:00,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18,0,1,0


### Seasonal features

The seasonal features will help our model understanding if there is any seasonality pattern followed by the data

In [10]:
compressors_all_data_hourly['datetime'] = pd.to_datetime(compressors_all_data_hourly['datetime'].fillna(method="ffill"))

In [11]:
compressors_all_data_hourly['hour'] = compressors_all_data_hourly['datetime'].dt.hour

In [12]:
compressors_all_data_hourly['day'] = compressors_all_data_hourly['datetime'].dt.day

In [13]:
compressors_all_data_hourly['month'] = compressors_all_data_hourly['datetime'].dt.month

Create columns in the dataframe

In [14]:
compressors_all_data_hourly['hour_sin'] = np.sin(compressors_all_data_hourly.hour*(2.*np.pi/24))
compressors_all_data_hourly['hour_cos'] = np.cos(compressors_all_data_hourly.hour*(2.*np.pi/24))

In [15]:
compressors_all_data_hourly['day_sin'] = np.sin((compressors_all_data_hourly.day)*(2.*np.pi/30))
compressors_all_data_hourly['day_cos'] = np.cos((compressors_all_data_hourly.day)*(2.*np.pi/30))

In [16]:
compressors_all_data_hourly['month_sin'] = np.sin((compressors_all_data_hourly.month)*(2.*np.pi/12))
compressors_all_data_hourly['month_cos'] = np.cos((compressors_all_data_hourly.month)*(2.*np.pi/12))

In [17]:
compressors_all_data_hourly.drop(['hour','day','month'] , axis = 1, inplace= True)

### Model split: Training, Validation

When working with time series as in this thesis, partitioning in training, validation, and testing must be done carefully to avoid overestimating the performance of the models. 

Predictive models do not have advanced knowledge of future time trends - in practice, such trends are likely to exist and have an adverse impact on model performance. To get an accurate assessment of the performance of a predictive model, it is recommended to perform training on older records and validation / testing using the newer records.

For both reasons, a time-dependent splitting strategy is an excellent choice for predictive maintenance models. The division is done by choosing a point in time according to the desired size of the training and test sets: all the records before the time point are used to train the model, and all the remaining records are used for testing.

In [18]:
compressors_all_data_hourly['datetime'].min()

Timestamp('2014-06-01 06:00:00')

In [19]:
compressors_all_data_hourly['datetime'].max()

Timestamp('2016-01-02 05:00:00')

In [20]:
# We establish the times corresponding to the records that will be used for training and tests.
threshold_dates = [
    pd.to_datetime("2015-09-30 01:00:00"), pd.to_datetime("2015-10-01 01:00:00")
]

In [21]:
# Create empty lists
test_results = []
models = []
total = len(threshold_dates)

In [22]:
# We make the partition of separate dates.
last_train_date = threshold_dates[0]
first_test_date = threshold_dates[1]

In [23]:
# Typically 20-30% of the data is used.
ntraining = compressors_all_data_hourly.loc[compressors_all_data_hourly["datetime"] < last_train_date]
ntesting = compressors_all_data_hourly.loc[compressors_all_data_hourly["datetime"] > first_test_date]
print(f"{ntraining.shape[0]} registers for trainning")
print(f"{ntesting.shape[0]} registers for testing")
print(f"{ntesting.shape[0] / ntraining.shape[0] * 100:0.1f}% of the total data is used for testing")

1165900 registers for trainning
223600 registers for testing
19.2% of the total data is used for testing


Check how many failures we have in the train and test

In [24]:
ntraining[ntraining['comp1_fail']==1.0]

,datetime,compressorID,cycle,comp1_maint,comp2_maint,comp3_maint,comp4_maint,current,rpm,pressure,...,age,model2,model3,model4,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos
6672,2015-03-06 06:00:00,1,6673,1.0,0.0,0.0,0.0,198.257975,456.862342,89.333995,...,18,0,1,0,1.0,6.123234e-17,9.510565e-01,0.309017,1.000000,6.123234e-17
20904,2015-03-19 06:00:00,2,6985,1.0,1.0,0.0,0.0,179.277874,322.388170,118.153934,...,7,0,0,1,1.0,6.123234e-17,-7.431448e-01,-0.669131,1.000000,6.123234e-17
33840,2015-02-06 06:00:00,3,6001,1.0,0.0,0.0,0.0,171.461890,431.490019,110.075130,...,8,0,1,0,1.0,6.123234e-17,9.510565e-01,0.309017,0.866025,5.000000e-01
48000,2015-02-16 06:00:00,4,6241,1.0,0.0,0.0,0.0,235.493717,488.515673,94.289697,...,7,0,1,0,1.0,6.123234e-17,-2.079117e-01,-0.978148,0.866025,5.000000e-01
52680,2015-08-30 06:00:00,4,10921,1.0,0.0,0.0,0.0,184.624797,486.118810,93.691194,...,7,0,1,0,1.0,6.123234e-17,-2.449294e-16,1.000000,-0.866025,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360752,2015-08-13 06:00:00,98,10513,1.0,0.0,0.0,1.0,172.192834,466.718730,97.213216,...,20,1,0,0,1.0,6.123234e-17,4.067366e-01,-0.913545,-0.866025,-5.000000e-01
1370064,2015-02-02 06:00:00,99,5905,1.0,0.0,0.0,0.0,174.266645,420.579042,110.596297,...,14,0,0,0,1.0,6.123234e-17,4.067366e-01,0.913545,0.866025,5.000000e-01
1372584,2015-05-18 06:00:00,99,8425,1.0,0.0,0.0,0.0,193.011248,447.278502,99.075683,...,14,0,0,0,1.0,6.123234e-17,-5.877853e-01,-0.809017,0.500000,-8.660254e-01
1384224,2015-02-12 06:00:00,100,6145,1.0,0.0,0.0,0.0,226.666914,484.516473,97.521897,...,5,0,0,1,1.0,6.123234e-17,5.877853e-01,-0.809017,0.866025,5.000000e-01


In [25]:
# Observe the shapes from the dataframes
fails_train = ntraining[ntraining["comp1_fail"] != 0].shape[0]
no_fails_train = ntraining[ntraining["comp1_fail"] == 0].shape[0]
fails_test = ntesting[ntesting["comp1_fail"] != 0].shape[0]
no_fails_test = ntesting[ntesting["comp1_fail"] == 0].shape[0]

print(f"{fails_train / no_fails_train * 100:0.3f}% of the total cases are failures in the train dataset")
print(f"{fails_test / no_fails_test * 100:0.3f}% of the total cases are failures in the test dataset")

0.013% of the total cases are failures in the train dataset
0.017% of the total cases are failures in the test dataset


We see it's crearly an unbalanced problem. 
Some of the techniques that are normally applied for unbalanced problems are:
- Minor sample of the majority class. 
- SMOTE (Synthetic Minority Over-sampling Technique). 
- Assign weights to minority class.

In a further stage of this project is planned to test these approaches in order to overcome the difficulties arisen due to the unbalanced data.

### Variables selection

We create lists containing the names of the columns for:
- Sensors
- Errors
- Maintenance
- Model
- Age
- Failures

In [26]:
# Iterate over the columns and get the names
cols_sensors = ['current' , 'rpm', 'pressure' , 'vibration']
cols_errors = [col for col in compressors_all_data_hourly.columns if (('error' in col) & ('count' not in col))]
cols_maint = [col for col in compressors_all_data_hourly.columns if "maint" in col]
cols_model_dum = [col for col in compressors_all_data_hourly.columns if 'model' in col]
cols_comp_info = [col for col in compressors_all_data_hourly.columns if 'age' in col]
cols_hour = [col for col in compressors_all_data_hourly.columns if 'hour' in col]
cols_day = [col for col in compressors_all_data_hourly.columns if 'day' in col]
cols_month = [col for col in compressors_all_data_hourly.columns if 'month' in col]

In [27]:
# Iterate over the columns and get the names
cols_failures_dum = [col for col in compressors_all_data_hourly.columns if 'fail' in col]

In [28]:
# Model target
target = cols_failures_dum
target

['comp1_fail', 'comp2_fail', 'comp3_fail', 'comp4_fail']

In [29]:
# Predictors to be included in the model
predictors = ['compressorID' , 'cycle']  + cols_sensors + cols_errors  + cols_maint  + cols_model_dum+ cols_comp_info + cols_hour + cols_day + cols_month
predictors

['compressorID',
 'cycle',
 'current',
 'rpm',
 'pressure',
 'vibration',
 'error1',
 'error2',
 'error3',
 'error4',
 'error5',
 'comp1_maint',
 'comp2_maint',
 'comp3_maint',
 'comp4_maint',
 'model2',
 'model3',
 'model4',
 'age',
 'hour_sin',
 'hour_cos',
 'day_sin',
 'day_cos',
 'month_sin',
 'month_cos']

In [30]:
# Categorical columns
categorical = cols_errors + cols_model_dum+ cols_comp_info
categorical

['error1',
 'error2',
 'error3',
 'error4',
 'error5',
 'model2',
 'model3',
 'model4',
 'age']

Remove NaN values in order to be possible running all the algorithms

In [31]:
# Remobe NaN and select only important columns for the model
ntraining = ntraining[(predictors+target)].dropna()
ntesting = ntesting[(predictors+target)].dropna()

In [32]:
#Create train and test dataframes
X_train = ntraining[predictors]
y_train = ntraining[target]
X_test = ntesting[predictors]
y_test = ntesting[target]

In [33]:
y_train.value_counts()

comp1_fail  comp2_fail  comp3_fail  comp4_fail
0.0         0.0         0.0         0.0           1165353
            1.0         0.0         0.0               165
1.0         0.0         0.0         0.0               138
0.0         0.0         0.0         1.0               120
                        1.0         0.0                91
            1.0         0.0         1.0                11
1.0         1.0         0.0         0.0                 7
0.0         1.0         1.0         0.0                 4
1.0         0.0         0.0         1.0                 4
                        1.0         0.0                 4
0.0         0.0         1.0         1.0                 3
dtype: int64

In [34]:
y_test.value_counts()

comp1_fail  comp2_fail  comp3_fail  comp4_fail
0.0         0.0         0.0         0.0           223429
            1.0         0.0         0.0               67
            0.0         0.0         1.0               35
1.0         0.0         0.0         0.0               34
0.0         0.0         1.0         0.0               26
            1.0         0.0         1.0                3
1.0         0.0         0.0         1.0                2
            1.0         0.0         0.0                2
0.0         0.0         1.0         1.0                1
1.0         0.0         1.0         0.0                1
dtype: int64

### Logistic Regression

As first MVP we will apply the Logistic Regression model and see how is the performance of the model.

As expected, our model has an abnormal behaviour and it's not able to identify any of the failures.

This performance is not acceptable.

In [35]:
# For each component

for comp in y_train.columns:
  logit = LogisticRegression()
  logit.fit(X_train,y_train[comp])
  y_pred = logit.predict(X_test)
  print(f"Classification matrix for: {comp}")
  print()
  print(confusion_matrix(y_test[comp], y_pred))
  print()


Classification matrix for: comp1_fail

[[223561      0]
 [    39      0]]

Classification matrix for: comp2_fail

[[223528      0]
 [    72      0]]

Classification matrix for: comp3_fail

[[223572      0]
 [    28      0]]

Classification matrix for: comp4_fail

[[223559      0]
 [    41      0]]



# Conclusion

The predictive maintenance problem is very imbalanced. Very small number of failures compared to the large amount of data collected by the sensors each hour. The logistic regression is not able to predict any of the failures. 

We will mofify the raw data accordingly performing some feature engineering.